In [0]:
%sql
SELECT 
    DATE(order_purchase_timestamp),
    SUM(price + freight_value) AS Total_price 
FROM eco.silver.orders_join
GROUP BY DATE(order_purchase_timestamp);

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT DATE(order_purchase_timestamp) AS date,
customer_state,customer_city,
 SUM(price + freight_value) AS State_Revenue
FROM eco.silver.geo_revenue
GROUP BY DATE(order_purchase_timestamp),customer_state,customer_city;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT DATE(order_purchase_timestamp) AS date,
    product_category_name_english, 
    COUNT(order_item_id) AS order_count
FROM eco.silver.category_revenue
GROUP BY product_category_name_english,DATE(order_purchase_timestamp);

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT DATE(order_purchase_timestamp) AS date, 
           SUM(price + freight_value) AS Daily_Revenue 
    FROM eco.silver.orders_join 
    GROUP BY DATE(order_purchase_timestamp)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    Customer_Category,DATE(order_purchase_timestamp),
    COUNT(customer_id) AS Customer_Count
FROM (
    SELECT 
        customer_id,DATE(order_purchase_timestamp),
        SUM(price + freight_value) AS Customer_Spent,
        CASE 
            WHEN SUM(price + freight_value) >= 2000 THEN 'High'
            WHEN SUM(price + freight_value) BETWEEN 501 AND 1999 THEN 'Mid'
            WHEN SUM(price + freight_value) BETWEEN 0 AND 500 THEN 'Low'
            ELSE 'Unknown'
        END AS Customer_Category
    FROM eco.silver.customer_spending
    GROUP BY DATE(order_purchase_timestamp),customer_id
) aggregated_data
GROUP BY DATE(order_purchase_timestamp),Customer_Category

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    product_name, DATE(order_purchase_timestamp),
    COUNT(order_item_id) AS order_count
FROM eco.silver.category_revenue
GROUP BY DATE(order_purchase_timestamp),product_name
order by order_count DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    DATE(order_purchase_timestamp),SUM(order_total) / COUNT(order_id) AS AOV
FROM (
    SELECT 
        DATE(order_purchase_timestamp),order_id,
        SUM(price + freight_value) AS order_total
    FROM eco.silver.orders_join
    GROUP BY DATE(order_purchase_timestamp),order_id
) order_totals
GROUP BY DATE(order_purchase_timestamp)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    ROUND(
        (SUM(CASE WHEN order_status = 'canceled' THEN 1 ELSE 0 END) * 100.0) / COUNT(order_id), 
        2
    ) AS Order_Return_Rate_Percentage,DATE(order_purchase_timestamp)
FROM eco.silver.orders_join
GROUP BY DATE(order_purchase_timestamp);

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT customer_id,Name, COUNT(order_id) AS total_orders,DATE(order_purchase_timestamp)
FROM eco.silver.customer_spending
GROUP BY customer_id,DATE(order_purchase_timestamp),Name
HAVING COUNT(order_id) > 1
order by total_orders DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    seller_company,
    seller_city,
    COUNT(order_item_id) AS total_items_sold
FROM eco.silver.order_seller
GROUP BY seller_company, seller_city
ORDER BY total_items_sold DESC
LIMIT 10;


In [0]:
%sql
SELECT 
    payment_type,
    COUNT(DISTINCT order_id) AS total_orders
FROM eco.silver.order_payments
where payment_type is not null
GROUP BY payment_type
ORDER BY total_orders DESC;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    CASE 
        WHEN payment_installments > 1 THEN 'EMI'
        ELSE 'One-Time Payment'
    END AS payment_type,
    COUNT(DISTINCT order_id) AS total_orders
FROM eco.silver.order_payments
WHERE payment_installments IS NOT NULL
GROUP BY 1
ORDER BY total_orders DESC;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    payment_type,  
    CASE 
        WHEN payment_installments > 1 THEN 'EMI'
        ELSE 'One-Time Payment' 
    END AS installment_type,
    COUNT(DISTINCT order_id) AS total_orders
FROM eco.silver.order_payments
WHERE payment_type IS NOT NULL AND order_id IS NOT NULL 
GROUP BY ROLLUP(payment_type, installment_type)
ORDER BY payment_type, 
         CASE WHEN installment_type = 'Total' THEN 1 ELSE 0 END, 
         total_orders DESC;
